## Bird Model

Input image에서 bird를 detect한 뒤, 해당 bird의 species를 구해 json 파일을 만들어 보냄

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import time
from tensorflow import keras
from keras.models import Model
from keras.models import load_model
import pandas as pd

In [3]:
detector = tf.saved_model.load("./detector").signatures['default']
classifier = load_model('bird_classifier.h5')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [4]:
df = pd.read_csv('species.csv')
species = df['Eng_name']
species

0      seagull
1    wildgoose
2        crane
3         ibis
Name: Eng_name, dtype: object

In [5]:
def run_detector(detector, path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    result = detector(converted_img)
    result = {key:value.numpy() for key, value in result.items()}

    birdbox = []
    for i in range(len(result["detection_boxes"])):
        if result["detection_class_entities"][i].decode("ascii") == "Bird" and result["detection_scores"][i] >= 0.1 :
            ymin, xmin, ymax, xmax = tuple(result["detection_boxes"][i])
            im_height, im_width, c = img.shape
            (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
            birdbox.append(img[int(top):int(bottom), int(left):int(right)])

    return birdbox

In [10]:
def run_classifier(classifier, birdbox):
    result = []
    for bird in birdbox:
        input = tf.image.resize(bird, [150, 150])
        input = np.array(input/np.amax(input)*255, np.int32)

        y = classifier.predict(input[None])
        result.append(species[np.argmax(y, axis=1)[0]])

    return list(set(result))

In [14]:
image_path = "./predict/predict(seagull).jpg" # temp

result = run_classifier(classifier, run_detector(detector, image_path))

In [15]:
result

['wildgoose', 'seagull', 'crane']

In [16]:
import json
from collections import OrderedDict

output_data = OrderedDict()
output_data["species"] = result

with open('./output/output.json', 'w', encoding="utf-8") as make_file:
    json.dump(output_data, make_file, ensure_ascii=False, indent="\t")